In [1]:
import logging
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%d-%b-%y %H:%M:%S',
                    level=logging.INFO
                   )

logging.info('Importing standard python libraries')
from pathlib import Path

logging.info('Importing third party python libraries')
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import cmocean.cm as cmo
import cycler
from scipy.special import erf  


logging.info('Setting paths')
base_path = Path('/work/n01/n01/fwg/irminger-proj')
raw_path = base_path / 'data/raw'
interim_path = base_path / 'data/interim'
processed_path = base_path / 'data/processed'
figure_path = base_path / 'figures'

logging.info('Setting plotting defaults')
# fonts
fpath = Path('/home/n01/n01/fwg/.local/share/fonts/PTSans-Regular.ttf')
if fpath.exists():
    font_prop = fm.FontProperties(fname=fpath)
    plt.rcParams['font.family'] = font_prop.get_family()
    plt.rcParams['font.sans-serif'] = [font_prop.get_name()]

# font size
matplotlib.use("pgf")
plt.rc('xtick', labelsize='8')
plt.rc('ytick', labelsize='8')
plt.rc('text', usetex=False)
plt.rcParams['axes.titlesize'] = 10
plt.rcParams["text.latex.preamble"] = "\\usepackage{euler} \\usepackage{paratype}  \\usepackage{mathfont} \\mathfont[digits]{PT Sans}"
plt.rcParams["pgf.preamble"] = plt.rcParams["text.latex.preamble"]
plt.rc('text', usetex=False)


n = 6
color = cmo.dense(np.linspace(0, 1,n))
plt.rcParams['axes.prop_cycle'] = cycler.cycler('color', color)

# output
dpi = 600
cm = 1/2.54
days = 24 * 60 * 60

logging.info("Opening ensemble")
ensemble_path = interim_path / "ensemble.zarr"
mld_path = processed_path / "mld.zarr"
wmt_path = processed_path / "enswmt.zarr"
assert mld_path.exists()


def select_marker(wind_duration):
    if np.allclose(wind_duration, 1.08e5):
        marker, colour = "*", "tab:blue"
    elif np.allclose(wind_duration, 2.16e5):
        marker, colour = "o", "tab:orange"
    elif np.allclose(wind_duration, 3.24e5):
        marker, colour = "^", "tab:green"
    elif np.allclose(wind_duration, 4.32e5):
        marker, colour = "s", "tab:red"
    elif np.allclose(wind_duration, 0):
        marker, colour = "D", "black"
    else:
        raise ValueError("wind_duration not as expected")
    return marker, colour



23-Feb-23 09:52:21 - Importing standard python libraries
23-Feb-23 09:52:21 - Importing third party python libraries
23-Feb-23 09:52:22 - Setting paths
23-Feb-23 09:52:22 - Setting plotting defaults
23-Feb-23 09:52:22 - Opening ensemble


In [26]:
#%matplotlib ipympl

In [2]:
ds_ensemble = xr.open_zarr(ensemble_path)
ds_mld = xr.open_zarr(mld_path).squeeze()
ds_mld = ds_mld.assign_coords({"wind_stress": ds_ensemble["wind_stress"],
                                "wind_duration": ds_ensemble["wind_duration"]})

ds_mld["tau_int"] = np.sqrt(2 * np.pi) \
                    * ds_mld["wind_stress"] * ds_mld["wind_duration"] \
                    * erf(10.5 * days / np.sqrt(2) \
                    / ds_mld["wind_duration"]) * 1e-5

delta_mld = ds_mld["mn_drop_depth"].isel(time=0) - ds_mld["mn_drop_depth"].isel(time=-1)

fig, axs = plt.subplots(1, 2, figsize=(6, 4.3))
ax = axs[0]
for run in ds_mld['run']:
    marker, colour = select_marker(ds_ensemble["wind_duration"].sel(run=run))
    
    ax.scatter(-ds_mld["tau_int"].sel(run=run),
                delta_mld.sel(run=run),
                marker=marker,
                color=colour)   

ax.ticklabel_format(axis="x", style="sci", scilimits=(0, 0), useMathText=True)

ax.scatter(None, None, marker="D", color="black", label=0)
ax.scatter(None, None, marker="*", color="tab:blue", label=1.08e5 / days)
ax.scatter(None, None, marker="o", color="tab:orange", label=2.16e5 / days)
ax.scatter(None, None, marker="^", color="tab:green", label=3.24e5 / days)
ax.scatter(None, None, marker="s", color="tab:red", label=4.32e5 / days)
ax.legend(title="Wind duration (days)")
ax.set_title("(a)", loc="left")

ax = axs[1]
for run in ds_mld['run']:
    marker, colour = select_marker(ds_ensemble["wind_duration"].sel(run=run))
    
    ax.scatter(-ds_mld["tau_int"].sel(run=run),
                delta_mld.sel(run=run),
                marker=marker,
                color=colour)   

ax.ticklabel_format(axis="x", style="sci", scilimits=(0, 0), useMathText=True)


ti = np.linspace(0.2, 10)
ax.plot(ti, (ti*7000) ** 0.5, c="k", label="$\\tau_{int}^{0.5}$")
ax.plot(ti, (ti*500) ** 0.54, c="k", ls="--", label="$\\tau_{int}^{0.54}$")
ax.set_ylim(10, None)

ax.set_xscale("log")
ax.set_yscale("log")
ax.set_title("(b)", loc="left")


fig.supxlabel("Integrated wind stress ($\\times$10$^{5}$ N$\\,$s$\\,$m$^{-2}$)", usetex=True)
fig.supylabel("$\Delta$ MLD (m)", usetex=True)

fig.suptitle("Change in mixed layer depth")
ax.legend()

fig.tight_layout()
fig.savefig(figure_path / "DeltaMLD.pdf")

In [33]:
delta_mld["tau_int"] = ds_mld["tau_int"]

In [51]:
x = np.log(-delta_mld["tau_int"]).values[1:]
y = np.log(delta_mld).values[1:]

/work/n01/n01/fwg/irminger-proj/irminger-proj/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: divide by zero encountered in log
  return func(*(_execute_task(a, cache) for a in args))


In [52]:
np.polyfit(x, y, 1)

array([0.53679872, 3.94793563])